In [1]:
import shutil
import os

dataset_path = "D:\\Job\\image_retrieval\\data\\COVID-19_Radiography_Dataset"
structured_dataset = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

# Function to structure the dataset
def create_image_dataset(source_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    classes = ["COVID", "Normal", "Viral Pneumonia", "Lung_Opacity"]

    for cls in classes:
        src_folder = os.path.join(source_dir, cls, "images")
        dst_folder = os.path.join(target_dir, cls)

        os.makedirs(dst_folder, exist_ok=True)

        for img in os.listdir(src_folder):
            src_path = os.path.join(src_folder, img)
            dst_path = os.path.join(dst_folder, img)
            shutil.copy(src_path, dst_path)  

# Run dataset structuring
create_image_dataset(dataset_path, structured_dataset)

print("Dataset structured successfully!")

Dataset structured successfully!


In [ ]:
from dataset import MedicalDataset, get_transforms
from torch.utils.data import DataLoader, random_split

dataset_root = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

full_dataset = MedicalDataset(dataset_root, transform=get_transforms())

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [3]:
from config import Config
from vit_fpn_rmac import ViT_FPN_RMAC
import torch
import torch.nn as nn

cfg = Config()
cfg.NUM_CLASSES = 4  

model = ViT_FPN_RMAC(cfg)
model.to("cuda")

optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


In [4]:
for epoch in range(cfg.NUM_EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0

    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        train_correct += (outputs.argmax(1) == y).sum().item()

    train_acc = train_correct / len(train_dataset)
    train_loss /= len(train_dataset)

    model.eval()
    val_loss, val_correct = 0.0, 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * x.size(0)
            val_correct += (outputs.argmax(1) == y).sum().item()

    val_acc = val_correct / len(val_dataset)
    val_loss /= len(val_dataset)

    print(f"Epoch {epoch+1}/{cfg.NUM_EPOCHS}")
    print(f"Train     Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val       Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")


Epoch 1/100
Train     Loss: 0.9196 | Acc: 0.6148
Val       Loss: 0.7157 | Acc: 0.7274
Epoch 2/100
Train     Loss: 0.6590 | Acc: 0.7397
Val       Loss: 0.5956 | Acc: 0.7682
Epoch 3/100
Train     Loss: 0.5167 | Acc: 0.8058
Val       Loss: 0.6953 | Acc: 0.7392
Epoch 4/100
Train     Loss: 0.4333 | Acc: 0.8402
Val       Loss: 0.5563 | Acc: 0.7867
Epoch 5/100
Train     Loss: 0.3796 | Acc: 0.8586
Val       Loss: 0.3894 | Acc: 0.8594
Epoch 6/100
Train     Loss: 0.3410 | Acc: 0.8741
Val       Loss: 0.3284 | Acc: 0.8805
Epoch 7/100
Train     Loss: 0.2892 | Acc: 0.8930
Val       Loss: 0.3289 | Acc: 0.8821
Epoch 8/100
Train     Loss: 0.2780 | Acc: 0.8951
Val       Loss: 0.3011 | Acc: 0.8909
Epoch 9/100
Train     Loss: 0.2595 | Acc: 0.9047
Val       Loss: 0.2913 | Acc: 0.8937
Epoch 10/100
Train     Loss: 0.2344 | Acc: 0.9132
Val       Loss: 0.2803 | Acc: 0.8949
Epoch 11/100
Train     Loss: 0.2202 | Acc: 0.9190
Val       Loss: 0.2671 | Acc: 0.9039
Epoch 12/100
Train     Loss: 0.2006 | Acc: 0.9249
Va

In [ ]:
#save model
model_save_path = "D:\\Job\\image_retrieval\\models\\covid_vit_fpn_rmac.pth"
torch.save(model.state_dict(), model_save_path)